### Notebook for creating spot instances

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [36]:
from aws_setup import *

In [9]:
name='fast-ai'

In [24]:
def allocate_vpc_addr(instance_id):
    ec2c = session.client('ec2')
#     alloc_addr = ec2c.allocate_address(Domain='vpc')
    alloc_addr = ec2c.describe_addresses()['Addresses'][0]
    addr_id = alloc_addr['AllocationId']
#     ec2c.associate_address(InstanceId=instance_id, AllocationId=addr_id)
    return alloc_addr

In [25]:

addr = allocate_vpc_addr('sdf')

In [4]:
ec2c = session.client('ec2')
# alloc_addr = ec2c.allocate_address(Domain='vpc')
# ec2c.associate_address(InstanceId=instance.id, AllocationId=alloc_addr['AllocationId'])

In [15]:
ec2c.describe_addresses()['Addresses'][0]

{'AllocationId': 'eipalloc-164bec2a',
 'Domain': 'vpc',
 'PublicIp': '52.43.199.214'}

In [6]:
alloc_id = ec2c.describe_addresses()['Addresses'][0]['AllocationId']

In [10]:
vpc_id, sg_id, subnet_id = get_vpc_ids(name)

In [ ]:
ec2.create_instances()

In [ ]:
def create_spot_instance(instance_type='t2.micro'):
    network_interface = ec2.create_network_interface(SubnetId=subnet_id, Groups=[sg_id])
    launch_specification = {
        'ImageId': ami, 
        'InstanceType': instance_type, 
        'KeyName': f'aws-key-{name}',
        'NetworkInterfaces': [{
            'DeviceIndex': 0, 
            'NetworkInterfaceId': network_interface.id
        }],
        'BlockDeviceMappings': [{
            'DeviceName': '/dev/sda1', 
            'Ebs': {
                # Volume size must be greater than snapshot size of 80
                'VolumeSize': 120, 
                'DeleteOnTermination': False,
    #             'DeleteOnTermination': True

                # SSD - use this to save money
    #             'VolumeType': 'gp2',

                # SSD io1 - superfast
                'VolumeType': 'io1',
                'Iops': 1
            }
        }]
    }
    spot_requests = ec2c.request_spot_instances(SpotPrice='0.5', LaunchSpecification=launch_specification)
    spot_request = spot_requests['SpotInstanceRequests'][0]
    instance_id = wait_on_fullfillment(spot_request)
    return instance_id

In [11]:
network_interface = ec2.create_network_interface(SubnetId=subnet_id, Groups=[sg_id])

In [ ]:
addr = allocate_vpc_addr(network_interface.id)

In [27]:
public_ip = addr['PublicIp']

In [37]:
ami = get_ami()

In [44]:
instance_type = 't2.micro'

In [52]:
launch_specification = {
    'ImageId': ami, 
    'InstanceType': instance_type, 
    'KeyName': f'aws-key-{name}',
    'NetworkInterfaces': [{
        'DeviceIndex': 0, 
        'NetworkInterfaceId': network_interface.id
    }],
    'BlockDeviceMappings': [{
        'DeviceName': '/dev/sda1', 
        'Ebs': {
            # Volume size must be greater than snapshot size of 80
            'VolumeSize': 120, 
            'DeleteOnTermination': False,
#             'DeleteOnTermination': True
            
            # SSD - use this to save money
#             'VolumeType': 'gp2',
            
            # SSD io1 - superfast
            'VolumeType': 'io1',
            'Iops': 1
        }
    }]
}

In [53]:
spot_requests = ec2c.request_spot_instances(SpotPrice='0.5', LaunchSpecification=launch_specification)

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
   'date': 'Wed, 28 Mar 2018 23:24:19 GMT',
   'server': 'AmazonEC2',
   'transfer-encoding': 'chunked',
   'vary': 'Accept-Encoding'},
  'HTTPStatusCode': 200,
  'RequestId': '5160e461-004c-43aa-aee1-9e0dd9c55193',
  'RetryAttempts': 0},
 'SpotInstanceRequests': [{'CreateTime': datetime.datetime(2018, 3, 28, 23, 24, 19, tzinfo=tzutc()),
   'InstanceInterruptionBehavior': 'terminate',
   'LaunchSpecification': {'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
      'Ebs': {'DeleteOnTermination': False,
       'Iops': 1,
       'VolumeSize': 120,
       'VolumeType': 'io1'}}],
    'ImageId': 'ami-8c4288f4',
    'InstanceType': 't2.micro',
    'KeyName': 'aws-key-fast-ai',
    'Monitoring': {'Enabled': False},
    'NetworkInterfaces': [{'DeviceIndex': 0,
      'NetworkInterfaceId': 'eni-febadad9'}],
    'Placement': {'AvailabilityZone': 'us-west-2b'},
    'SecurityGroups': [{'GroupName': 'default'}]},
   '

In [57]:
spot_request = spot_requests['SpotInstanceRequests'][0]

In [58]:
req_id = spot_request['SpotInstanceRequestId']

In [60]:
req_id

'sir-7mcrbnap'

In [68]:
def wait_on_fullfillment(req_status):
    while req_status['State'] != 'closed':
        print('Waiting on spot fullfillment...')
        sleep(5)
        req_statuses = ec2c.describe_spot_instance_requests(Filters=[{'Name': 'spot-instance-request-id', 'Values': [req_status['SpotInstanceRequestId']]}])
        req_status = req_statuses['SpotInstanceRequests'][0]
        if req_status['State'] == 'failed':
            print('Spot instance request failed:', req_status['Status'])
            return None
    instance_id = req_status['InstanceId']
    print('Fullfillment completed. InstanceId:', instance_id)
    return instance_id
    
    

In [69]:
req_statuses = ec2c.describe_spot_instance_requests(Filters=[{'Name': 'spot-instance-request-id', 'Values': ['sir-7mcrbnap']}])
req = req_statuses['SpotInstanceRequests'][0]



In [72]:
instance_id = wait_on_fullfillment(req)

Fullfillment completed. InstanceId: i-06bfb34e49cec49e1


In [76]:
instance = list(ec2.instances.filter(Filters=[{'Name': 'instance-id', 'Values': [instance_id]}]).all())[0]

In [75]:
list(instance.all())

[ec2.Instance(id='i-06bfb34e49cec49e1')]

In [71]:
list(ec2.instances.all())

[ec2.Instance(id='i-06bfb34e49cec49e1')]

In [77]:
instance.public_ip_address

ec2.Instance(id='i-06bfb34e49cec49e1')

### Attach DFS

### Adapting code shell

In [ ]:

export networkInterfaceId=`aws ec2 create-network-interface --subnet-id $subnetId --groups $securityGroupId --query 'NetworkInterface.NetworkInterfaceId' --output text`
export allocAddr=`aws ec2 allocate-address --domain vpc --query 'AllocationId' --output text`
export assocId=`aws ec2 associate-address --network-interface-id $networkInterfaceId --allocation-id $allocAddr --query 'AssociationId' --output text`
export instancePublicIp=`aws ec2 describe-addresses --query 'Addresses[?AssociationId==\`'$assocId'\`][PublicIp]' --output text`

export spotInstanceRequestId=`aws ec2 request-spot-instances --spot-price "$2" --launch-specification '{"ImageId": "'$ami'", "InstanceType": "'$1'", "KeyName": "'aws-key-$name'", "NetworkInterfaces": [{"DeviceIndex": 0, "NetworkInterfaceId": "'$networkInterfaceId'"}], "BlockDeviceMappings": [{"DeviceName": "/dev/sda1", "Ebs": {"VolumeSize": 20, "VolumeType": "gp2", "DeleteOnTermination": true}}]}' --query 'SpotInstanceRequests[0].[SpotInstanceRequestId]' --output text`
